In [2]:
#importing libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
#import dataset
ds= pd.read_csv('cleand_coffee_data.csv')
ds= ds.loc[: , ~ds.columns.str.contains('^Unnamed')]


In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Size         767 non-null    object 
 1   Coffee_Type  767 non-null    object 
 2   Milk_Type    767 non-null    object 
 3   Flavor       767 non-null    object 
 4   Shots        767 non-null    float64
 5   Temperature  767 non-null    float64
 6   Cup_Quality  767 non-null    int64  
 7   Price        767 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 48.1+ KB


In [5]:
ds.columns

Index(['Size', 'Coffee_Type', 'Milk_Type', 'Flavor', 'Shots', 'Temperature',
       'Cup_Quality', 'Price'],
      dtype='object')

In [6]:
#splitting dataset in Independent and Depedent variables
x=ds[['Size', 'Coffee_Type', 'Milk_Type', 'Flavor', 'Shots', 'Temperature',
       'Cup_Quality']]
y=ds[['Price']]

In [7]:
#encodeing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe= OneHotEncoder()
ohe.fit(x[['Size', 'Coffee_Type', 'Milk_Type', 'Flavor']])

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [8]:
ct= make_column_transformer((OneHotEncoder(handle_unknown='ignore',categories= ohe.categories_),['Size', 'Coffee_Type', 'Milk_Type', 'Flavor']),
                           remainder='passthrough', force_int_remainder_cols=False)
ct

,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,False
,categories,"[array(['Large... dtype=object), array(['Arabi... dtype=object), ...]"
,drop,None
,sparse_output,True


In [9]:
#make pipeline
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
reg= LinearRegression()

In [10]:
pipe= make_pipeline(ct,reg)
pipe

,steps,"[('columntransformer', ...), ('linearregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [11]:
#checking accoracy
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
scores=[]
for i in range(0,101):
    x_train,x_test,y_train,y_test= train_test_split(x,y ,test_size=0.25, random_state=i)
    pipe.fit(x_train,y_train)
    res= pipe.predict(x_test)
    score=r2_score(y_test,res)
    scores.append(score)

In [12]:
bestindex= np.argmax(scores)
scores[bestindex]

0.9920289957431891

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y ,test_size=0.25, random_state=bestindex)
pipe.fit(x_train,y_train)

,steps,"[('columntransformer', ...), ('linearregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:
ds.columns

Index(['Size', 'Coffee_Type', 'Milk_Type', 'Flavor', 'Shots', 'Temperature',
       'Cup_Quality', 'Price'],
      dtype='object')

In [25]:
Size= input('Enter size of coffee : ')
Coffee_Type= input('Enter type of coffee : ')
Milk_Type= input('Enter type of milk : ')
Flavor= input('Enter flavour of coffee : ')
Shots =float(input('Enter no of shots : '))
Temperature= float(input('Enter tempreture of coffee : '))
Cup_Quality= int(input('Enter quality of cup : '))

myip= pd.DataFrame(data=[[Size, Coffee_Type, Milk_Type, Flavor, Shots, Temperature,
       Cup_Quality]], columns=['Size', 'Coffee_Type', 'Milk_Type', 'Flavor', 'Shots', 'Temperature',
       'Cup_Quality'])
res = pipe.predict(myip)
print(f'You should buy it for ₹{abs(round(res[0,0]}')

Enter size of coffee :  Large
Enter type of coffee :  Liberica
Enter type of milk :  Soy
Enter flavour of coffee :  Vanilla
Enter no of shots :  3.0
Enter tempreture of coffee :  83.56
Enter quality of cup :  5


You should buy it for ₹[[18.78091531]]


In [16]:
import pickle as pkl
pkl.dump(pipe, open('CoffeePricePredictionApp.pkl','wb'))